In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 15.2.1 여러 개의 입력 또는 컬러 채널 다루기
이미지 파일 읽기

In [ ]:
import tensorflow as tf

In [ ]:
img_raw = tf.io.read_file('example-image.png')

NotFoundError: ignored

In [ ]:
img = tf.image.decode_image(img_raw)
print('이미지 크기:', img.shape)

In [ ]:
import imageio

In [ ]:
img = imageio.imread('example-image.png')

In [ ]:
print('이미지 크기:', img.shape)
print('채널 개수:', img.shape[2])
print('이미지 데이터 타입:', img.dtype)

## 15.2.2 드롭아웃으로 신경망 규제

### L2 패널티 추가

In [ ]:
from tensorflow import keras

In [ ]:
conv_layer = keras.layers.Conv2D(
    filters=16,
    kernel_size=(3,3),
    kernel_regularizer=keras.regularizers.l2(0.001)
)
fc_layer = keras.layers.Dense(
    units=16,
    kernel_regularizer=keras.regularizers.l2(0.001)
)

### 드롭아웃
뒤쪽 층의 은닉 유닛에 적용 \
신경망 훈련 동안 반복마다 은닉 유닛의 일부가 확률 $p_{drop}$ 만큼 랜덤하게 드롭아웃
$p_{keep} = 1 = p_{drop}$ 만큼 랜덤하게 일부가 유지

## 15.2.3 분류를 위한 손실 함수

In [ ]:
import tensorflow_datasets as tfds
### 이진 크로스 엔트로피
bce_probas = tf.keras.losses.BinaryCrossentropy(from_logits=False)
bce_logits = tf.keras.losses.BinaryCrossentropy(from_logits=True)
logits = tf.constant([0.8])
probas = tf.keras.activations.sigmoid(logits)
tf.print(
    'BCE 확률: {:.4f}'.format(
      bce_probas(y_true=[1], y_pred=probas)),
      '로짓: {:.4f}'.format(
          bce_logits(y_true=[1], y_pred=logits)))

### 범주형 크로스 엔트로피
cce_probas = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
cce_logits = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
logits = tf.constant([1.5, 0.8, 2.1])
probas = tf.keras.activations.sigmoid(logits)
tf.print(
    'CCE 확률: {:.4f}'.format(
      cce_probas(y_true=[0, 0, 1], y_pred=probas)),
      '로짓: {:.4f}'.format(
          cce_logits(y_true=[0, 0, 1], y_pred=logits)))

### 희소 범주형 크로스 엔트로피
sp_cce_probas = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
sp_cce_logits = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
tf.print(
    'CCE 확률: {:.4f}'.format(
      sp_cce_probas(y_true=[2], y_pred=probas)),
      '로짓: {:.4f}'.format(
          sp_cce_logits(y_true=[2], y_pred=logits)))

# 15.3 텐서플로 사용하여 심층 합성곱 신경망 구현

### 데이터 적재, 전처리

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
## 데이터 적재
mnist_bldr = tfds.builder('mnist')
mnist_bldr.download_and_prepare()
datasets = mnist_bldr.as_dataset(shuffle_files=False)
mnist_train_orig = datasets['train']
mnist_test_orig = datasets['test']

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20

In [ ]:
mnist_train = mnist_train_orig.map(
    lambda item: (tf.cast(item['image'], tf.float32)/255,
                  tf.cast(item['label'], tf.int32)))
mnist_test = minst_test_orig.map(
    lambda item: (tf.cast(item['image'], tf.float32)/255,
                  tf.cast(item['label'], tf.int32)))

tf.random.set_seed(1)

mnist_train = mnist_train.shuffle(buffer_size = BUFFER_SIZE,
                                  reshuffle_each_iteration=False)
mnist_valid = mnist_train.take(10000).batch(BATCH_SIZE) # 검증 데이터 
mnist_train = mnist_train.skip(10000).batch(BATCH_SIZE) # 학습 데이터

### 텐서플로 케라스 API를 사용해 CNN 구현
tf.keras.layers.Conv2D : **2차원 합성곱**
* filters
* kernel_size
* strides
* padding

tf.keras.layers.MaxPool2D, tf.keras.layers.AvgPool2D : **서브샘플링**
* pool_size
* strides
* padding

tf.keras.layers.Dropout : **드롭아웃 수행**
* rate







In [ ]:
## 합성곱과 풀링층을 모델에 추가
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), 
                                strides=(1, 1), padding='same', 
                                data_format='channels_last', 
                                name='conv_1', activation='relu'))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), name='pool_1'))

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(5, 5), 
                                strides=(1, 1), padding='same', 
                                data_format='channels_last', 
                                name='conv_2', activation='relu'))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), name='pool_2'))

In [ ]:
## 입력 크기에 대한 출력 크기
model.compute_output_shape(input_shape=(16, 28, 28, 1))

In [ ]:
## 밀집 층(완전 연결 층)
model.add(tf.keras.layers.Flatten())

In [ ]:
model.compute_output_shape(input_shape=(16, 28, 28, 1))

In [ ]:
model.add(tf.keras.layers.Dense(units=1024, name='fc_1',
                                activation='relu'))

model.add(tf.keras.layers.Dropout(rate=0.5))

## 클래스 레이블 10개에 대응하는 출력 유닛
model.add(tf.keras.layers.Dense(units=10, name='fc_2',
                                activation='softmax'))

In [ ]:
## 분류를 위한 손실 함수
### 정수(희소한) 레이블을 사용하는 다중 분류
tf.random.set_seed(1)
model.build(input_shape=(None, 28, 28, 1))

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

In [ ]:
history = model.fit(mnist_train, epochs=NUM_EPOCHS,
                    validation_data=mnist_valid,
                    shuffle=True)

In [ ]:
hist = history.history
x_arr = np.arange(len(hist['loss'])) + 1

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, hist['loss'], '-o', label='Train loss')
ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss')
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Loss', size=15)
ax.legend(fontsize=15)
ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, hist['accuracy'], '-o', label='Train acc.')
ax.plot(x_arr, hist['val_accuracy'], '--<', label='Validation acc.')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Accuracy', size=15)

plt.show()

In [ ]:
test_results = model.evaluate(mnist_test.batch(20))
print('테스트 정확도: {:.2f}\%'.format(test_results[1]*100))

In [ ]:
batch_test = next(iter(mnist_test.batch(12)))
preds = model(batch_test[0])
tf.print(preds.shape)

In [ ]:
preds = tf.argmax(preds, axis=1)
print(preds)

In [ ]:
fig = plt.figure(figsize=(12, 4))
for i in range(12):
  ax = fig.add_subplot(2, 6, i+1)
  ax.set_xticks([]); ax.set_yticks([])
  img = batch_test[0][i, :, :, 0]
  ax.imshow(img, cmap='gray_r')
  ax.text(0.9, 0.1, '{}'.format(preds[i]), size=15, color='blue',
          horizontalalignment='center',
          verticalalignment='center',
          transform=ax.transAxes)
  
plt.show()